## Importing own implementations

In [1]:
import Classes.LDA as LDA
import Classes.NaiveBayes as NaiveBayes

## Other imports

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import scipy.optimize as optimize
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import CategoricalNB
import math
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC

# Ignore unwanted warnings
import warnings
warnings.filterwarnings('ignore')
# ignore some warnings about some scipy function getting changed in the next update
warnings.simplefilter(action='ignore', category=FutureWarning)

/Users/andrei/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Reading the dataset

In [3]:
path_train = "Datasets/fashion_train.npy"
path_test = "Datasets/fashion_test.npy"

train = np.load(path_train)
test = np.load(path_test)

train_percentage = int((train.shape[0]) * 0.7)
train_percentage

#Split the training and test data into features and labels
X_train = train[:train_percentage,:784]
y_train = train[:train_percentage,784]

X_val = train[train_percentage:, :784]
y_val = train[train_percentage:, 784]

X_test = test[:,:784]
y_test = test[:,784]

X_train = X_train/255
X_val = X_val/255
X_test = X_test/255

## Declaring our own classes

In [4]:
ourLDA = LDA.ourLDA()
ourNaiveBayes = NaiveBayes.ourNaiveBayes()

## Getting the first two liniar discriminants for both the train and the test sets

In [5]:
ourLDA.set_lda_components(X_train, y_train)
X_lda = ourLDA.get_two_discriminants(X_train)
X_lda = X_lda.real
X_lda_test = ourLDA.get_two_discriminants(X_test)
X_lda_test = X_lda_test.real

### Scale

In [6]:
scaler = StandardScaler()
X_lda_std = scaler.fit_transform(X_lda)
X_lda_test_std = scaler.fit_transform(X_lda_test)

## Setting up Naive Bayes

In [7]:
ourNaiveBayes.set_priors(X_lda, y_train)
ourNaiveBayes.set_bin_number(5)
ourNaiveBayes.set_conditionals(X_lda, y_train)

### Making predictions with Naive Bayes

In [8]:
predictions_lda = ourNaiveBayes.predict_class(X_lda_test)
print("Accuracy of:", (np.sum(predictions_lda == y_test))/(X_lda_test.shape[0])*100, "%")

Accuracy of: 67.9 %


# KNN

### KNN 5 neighbors

In [9]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_lda, y_train)
pred_knn = knn.predict(X_lda)
print("Accuracy of:", accuracy_score(pred_knn, y_train)*100, "%")

Accuracy of: 82.75714285714287 %


### KNN 3 neighbors

In [10]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_lda, y_train)
pred_knn = knn.predict(X_lda)
print("Accuracy of:", accuracy_score(pred_knn, y_train)*100, "%")

Accuracy of: 84.92857142857143 %


## Standardized features

### KNN 5 neighbors

In [11]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_lda_std, y_train)
pred_knn = knn.predict(X_lda_std)
print("Accuracy of:", accuracy_score(pred_knn, y_train)*100, "%")

Accuracy of: 82.8 %


### KNN 3 neighbors

In [12]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_lda_std, y_train)
pred_knn = knn.predict(X_lda_std)
print("Accuracy of:", accuracy_score(pred_knn, y_train)*100, "%")

Accuracy of: 84.61428571428571 %


## Using grid search to find optimal parameters

In [13]:
knn = KNeighborsClassifier()

param_grid = {'n_neighbors': [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]}
grid_search = GridSearchCV(knn, param_grid, cv=5, scoring="accuracy")
grid_search.fit(X_lda, y_train)

print("Best k value:", grid_search.best_params_['n_neighbors'])
print("Best accuracy:", grid_search.best_score_*100, "%")

Best k value: 17
Best accuracy: 79.0 %


In [14]:
knn = KNeighborsClassifier(n_neighbors=grid_search.best_params_['n_neighbors'])
knn.fit(X_lda, y_train)
pred_knn = knn.predict(X_lda)
print("Accuracy of:", accuracy_score(pred_knn, y_train)*100, "%")

Accuracy of: 80.45714285714286 %


## Testing accuracy for chosen model on the test set

In [15]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_lda_std, y_train)
pred_knn = knn.predict(X_lda_test_std)
print("Accuracy of:", accuracy_score(pred_knn, y_test)*100, "%")

Accuracy of: 70.72 %


## Setting up SVM

### RBF kernel

In [16]:
svm = make_pipeline(StandardScaler(), SVC(gamma='auto'))
svm.fit(X_lda, y_train)
pred_svm = svm.predict(X_lda)
print("Accuracy of:", accuracy_score(pred_svm, y_train)*100, "%")

Accuracy of: 79.81428571428572 %


### Polynomial kernel

In [17]:
svm = make_pipeline(StandardScaler(), SVC(gamma='auto', kernel='poly', degree=3))
svm.fit(X_lda, y_train)
pred_svm = svm.predict(X_lda)
print("Accuracy of:", accuracy_score(pred_svm, y_train)*100, "%")

Accuracy of: 76.45714285714286 %


### Finiding optimal parameters with grid search

In [18]:
svm = SVC(gamma='auto')

param_grid_svm = {'gamma': ['auto', 'scale'], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid']}
grid_search_svm = GridSearchCV(svm, param_grid_svm, cv=5, scoring="accuracy")
grid_search_svm.fit(X_lda_std, y_train)

print("Best gamma value:", grid_search_svm.best_params_['gamma'])
print("Best kernel value:", grid_search_svm.best_params_['kernel'])
print("Best accuracy:", grid_search_svm.best_score_*100, "%")

svm = make_pipeline(StandardScaler(), SVC(gamma=grid_search_svm.best_params_['gamma'], kernel=grid_search_svm.best_params_['kernel']))
svm.fit(X_lda, y_train)
pred_svm = svm.predict(X_lda)
print("Accuracy of:", accuracy_score(pred_svm, y_train)*100, "%")

Best gamma value: auto
Best kernel value: linear
Best accuracy: 79.72857142857143 %
Accuracy of: 79.85714285714286 %


## Testing accuracy for chosen model on the test set

In [19]:
svm = SVC(gamma='auto')
svm.fit(X_lda, y_train)
pred_svm = svm.predict(X_lda_test)
print("Accuracy of:", accuracy_score(pred_svm, y_test)*100, "%")

Accuracy of: 42.32 %


## Setting up Tensorflow

In [21]:
X_test = X_test.reshape((5000, 28, 28, 1))
X_val = X_val.reshape((3000, 28, 28, 1))
X_train = X_train.reshape((7000, 28, 28, 1))

CNN = tf.keras.Sequential(
    [
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation="relu",input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Conv2D(48, (3,3), padding='same', activation="relu"),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation="relu"),
    tf.keras.layers.MaxPooling2D((2, 2), strides=2),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(100, activation="relu"),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(5, activation="linear")
]
)

CNN.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

callbacks = [EarlyStopping(patience=3, monitor="val_loss")]
CNN.fit(X_train,y_train, epochs=60, validation_data=(X_val, y_val), callbacks=callbacks)

Epoch 1/60
219/219 [==============================] - 2s 9ms/step - loss: 1.3846 - accuracy: 0.3960 - val_loss: 1.0620 - val_accuracy: 0.6167
Epoch 2/60
219/219 [==============================] - 2s 10ms/step - loss: 1.0121 - accuracy: 0.5583 - val_loss: 0.8831 - val_accuracy: 0.6470
Epoch 3/60
219/219 [==============================] - 2s 11ms/step - loss: 0.8990 - accuracy: 0.6261 - val_loss: 0.8162 - val_accuracy: 0.6847
Epoch 4/60
219/219 [==============================] - 2s 10ms/step - loss: 0.8123 - accuracy: 0.6694 - val_loss: 0.7210 - val_accuracy: 0.7167
Epoch 5/60
219/219 [==============================] - 2s 11ms/step - loss: 0.7430 - accuracy: 0.6943 - val_loss: 0.6641 - val_accuracy: 0.7283
Epoch 6/60
219/219 [==============================] - 2s 10ms/step - loss: 0.6909 - accuracy: 0.7159 - val_loss: 0.6313 - val_accuracy: 0.7390
Epoch 7/60
219/219 [==============================] - 2s 10ms/step - loss: 0.6535 - accuracy: 0.7319 - val_loss: 0.5952 - val_accuracy: 0.7500


### Making predictions with Tesorflow

In [25]:
logits_test = CNN(X_test)
probs_test = tf.nn.softmax(logits_test)
probs_test = tf.make_tensor_proto(probs_test)
probs_np_test = tf.make_ndarray(probs_test)
y_pred = np.argmax(probs_np_test, axis=1)
print("Accuracy of:", accuracy_score(y_pred, y_test)*100, "%")

Accuracy of: 87.2 %
